In [180]:
import torch
from torch.nn import MultiheadAttention
from transformers import LlamaModel, LlamaConfig,LlamaForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import torch.nn as nn 
#if the directory is not the root of the project, change it
os.chdir('/home/mila/e/emiliano.penaloza/RLPHF')
import LoRA.loralib  as lora
os.environ['TRANSFORMERS_CACHE'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['HF_HOME'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['HF_DATASETS_CACHE'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['TORCH_HOME'] = '/home/mila/e/emiliano.penaloza/scratch/models'
cache_dir = '/home/mila/e/emiliano.penaloza/scratch/models'



In [181]:
from transformers import BertTokenizer, BertModel

r = 1
alpha = 0.1
dropout = 0.1
n_transformer_layers = 1
n_transformer_heads = 2
d_emb = 64
d_model = 16
a_b = 2 
kvq =3
#this is to produce a layer at a time
output_dim = (r *  d_model ) * a_b  * kvq   
a = 16 

#load bert as the model 
model = BertModel.from_pretrained('bert-base-uncased',cache_dir  = cache_dir).to(0)
A = model.config.hidden_size

hypernet = HyperNet(r, alpha, dropout, output_dim,d_emb, d_model, n_transformer_layers, n_transformer_heads,a = a ).to(0)
controller = HyperNetController(hypernet,target_modules = ['query','key','value'], 
                                r = r,
                                A = A,
                                a = a)


controller.augmentLLM(model)



KeyboardInterrupt: 

In [118]:
# Create a random task embedding with dimension user_emb_dim
bsize  = 10 
d_emb_user = 64
n_layers = 12
task_embedding = torch.randn((bsize,n_layers,(d_emb_user))).to(0)
print(f"{task_embedding.shape=}")

# Create an identity matrix with dimensions matching the batch size and d_emb
layers = torch.eye(d_model ).unsqueeze(0).repeat(bsize, 1, 1)[:,:n_layers,:].to(0)
print(f"{layers.shape=}")
# new_layers = hypernet(task_embedding,layers).view(bsize,n_layers*kvq,r,-1)
print(f"{new_layers.shape=}")

task_embedding.shape=torch.Size([10, 12, 64])
layers.shape=torch.Size([10, 12, 16])
new_layers.shape=torch.Size([10, 36, 1, 32])


In [ ]:
#feed something in laguage to the bert model b

input_ids = torch.randint(0, 1000, (bsize, 20)).to(0)
attention_mask = torch.ones_like(input_ids).to(0)
# output = model(input_ids,attention_mask = attention_mask)

In [14]:
output.pooler_output.shape,output.last_hidden_state.shape

(torch.Size([10, 768]), torch.Size([10, 20, 768]))

In [35]:
sent = load_dataset("nyu-mll/glue", "sst2")
mrpc = load_dataset("nyu-mll/glue", "mrpc")

In [36]:
sent

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [37]:
mrpc

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [155]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the datasets
sst2 = load_dataset("nyu-mll/glue", "sst2")
mrpc = load_dataset("nyu-mll/glue", "mrpc")

# Function to process MRPC dataset
def process_mrpc(example):
    return {
        "text": example["sentence1"] + " " + example["sentence2"],
        "label": example["label"],
        "dataset": "mrpc"
    }

# Process MRPC dataset (only the 'train' split)
mrpc_processed = mrpc['train'].map(process_mrpc)

# Function to process SST2 dataset
def process_sst2(example):
    return {
        "text": example["sentence"],
        "label": example["label"],
        "dataset": "sst2"
    }

# Process SST2 dataset (only the 'train' split)
sst2_processed = sst2['train'].map(process_sst2)

# Convert both datasets to pandas DataFrames
df_mrpc = mrpc_processed.to_pandas()
df_sst2 = sst2_processed.to_pandas()

# Calculate the number of samples to keep for SST2
n_mrpc = len(df_mrpc)
n_sst2_to_keep = int(n_mrpc * 1.5)  # 60% SST2, 40% MRPC

# Downsample SST2
df_sst2_downsampled = df_sst2.sample(n=n_sst2_to_keep, random_state=42)

# Combine the DataFrames
df_combined = pd.concat([df_mrpc, df_sst2_downsampled], ignore_index=True)

# Ensure all columns have the same dtype
df_combined["text"] = df_combined["text"].astype(str)
df_combined["label"] = df_combined["label"].astype(int)
df_combined["dataset"] = df_combined["dataset"].astype(str)

# Create a stratified train-test split
train_df, test_df = train_test_split(df_combined, test_size=0.2, stratify=df_combined['dataset'], random_state=42)

# Shuffle the datasets
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Convert back to Hugging Face datasets
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Print info about the final datasets
print("Train dataset:")
print(train_dataset)
print("\nTest dataset:")
print(test_dataset)

# Verify the distribution of tasks in each split
print("\nTask distribution in train set:")
print(train_df['dataset'].value_counts(normalize=True))
print("\nTask distribution in test set:")
print(test_df['dataset'].value_counts(normalize=True))


#subset the train_dataset to only have text label and dataset 
train_dataset = train_dataset.remove_columns(['idx'])
test_dataset = test_dataset.remove_columns(['idx'])

train_dataset = train_dataset.remove_columns(['sentence1','sentence2','sentence'])
test_dataset = test_dataset.remove_columns(['sentence1','sentence2','sentence'])

Train dataset:
Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'text', 'dataset', 'sentence'],
    num_rows: 7336
})

Test dataset:
Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'text', 'dataset', 'sentence'],
    num_rows: 1834
})

Task distribution in train set:
dataset
sst2    0.600055
mrpc    0.399945
Name: proportion, dtype: float64

Task distribution in test set:
dataset
sst2    0.599782
mrpc    0.400218
Name: proportion, dtype: float64


In [156]:
#laod train_df into a dataloader 
from torch.utils.data import DataLoader
from transformers import BertTokenizer

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [157]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Define the model name
model_name = "bert-base-uncased"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Model loaded: {model_name}")
print(f"Model is on device: {device}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded: bert-base-uncased
Model is on device: cuda


In [158]:
class_encodings = nn.Parameter(torch.empty(2,64))

nn.init.xavier_normal_(class_encodings)
print('initialized')

initialized


In [171]:
from transformers import BertTokenizer, BertModel

r = 1
alpha = 0.1
dropout = 0.1
n_transformer_layers = 1
n_transformer_heads = 2
d_emb = 64
d_model = 16
a_b = 2 
kvq =3
#this is to produce a layer at a time
output_dim = (r *  d_model ) * a_b  * kvq   
a = 16 

#load bert as the model 
# model = BertModel.from_pretrained('bert-base-uncased',cache_dir  = cache_dir).to(0)
A = model.config.hidden_size

hypernet = HyperNet(r, alpha, dropout, output_dim,d_emb, d_model, n_transformer_layers, n_transformer_heads,a = a ).to(0)
controller = HyperNetController(hypernet,target_modules = ['query','key','value'], 
                                r = r,
                                A = A,
                                a = a)


controller.augmentLLM(model)

controller.freezeParams(model,False)

model_trainable_params = [param for param in model.parameters() if param.requires_grad]
modelOptimizer = torch.optim.Adam(model_trainable_params, lr=1e-4)
hypernetOptimizer = torch.optim.Adam(hypernet.parameters(), lr=1e-4)

In [172]:
model.train()
hypernet.train()    
print('training mode')

training mode


In [179]:
task_mapper = {'mrpc': 0 , 'sst2': 1}
device = 'cuda'
for b in train_loader:
    modelOptimizer.zero_grad()
    hypernetOptimizer.zero_grad()
    encodings = tokenizer(b['text'], truncation=True, padding=True,return_tensors='pt')
    labels = b['label'].to(device)
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)
    tasks = [task_mapper[task] for task in b['dataset']]

    task_embedding = torch.stack([class_encodings[task] for task in tasks]).unsqueeze(1).repeat(1,12,1).to(device)
    identity_matrix = torch.eye(d_model).unsqueeze(0).repeat(input_ids.shape[0], 1, 1).to(device)[:,:12,:]
    hypernet_layers = hypernet(task_embedding,identity_matrix).view(bsize,n_layers*kvq,r,-1)
    controller.updateLayers(hypernet_layers)
    
    loss  = model(input_ids,attention_mask = attention_mask,labels = labels).loss
    #optimizer stuff 
    loss.backward()
    modelOptimizer.step()
    hypernetOptimizer.step()


In [177]:
model(input_ids,attention_mask = attention_mask,labels = labels)

SequenceClassifierOutput(loss=tensor(0.7819, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.3513,  0.2113],
        [-0.5490,  0.0690],
        [-0.2109, -0.1376],
        [-0.2421,  0.0646],
        [-0.4282,  0.3292],
        [-0.2840,  0.3000],
        [-0.6149,  0.1521],
        [-0.2951,  0.0040],
        [-0.2611,  0.1063],
        [-0.3548,  0.2396],
        [-0.3537,  0.0051],
        [-0.4265,  0.2425],
        [-0.3525,  0.2373],
        [-0.4346,  0.1305],
        [-0.2350,  0.0970],
        [-0.3173, -0.0028],
        [-0.5806,  0.0109],
        [-0.3009, -0.0507],
        [-0.0360, -0.0360],
        [-0.1475,  0.1545],
        [-0.5010,  0.2160],
        [-0.2557,  0.1231],
        [-0.5865,  0.2402],
        [-0.4791,  0.0761],
        [-0.4312,  0.1473],
        [-0.2364,  0.2963],
        [-0.5053,  0.1519],
        [-0.4846,  0.1573],
        [-0.3797,  0.0935],
        [-0.0753,  0.2044],
        [ 0.0846,  0.1572],
        [-0.2918,  0.2107]], devic

In [174]:
#print trainable parameters
for name, param in hypernet.named_parameters():
    if param.requires_grad:
        print(name)

transformer_encoder.layers.0.self_attn.in_proj_weight
transformer_encoder.layers.0.self_attn.in_proj_bias
transformer_encoder.layers.0.self_attn.out_proj.weight
transformer_encoder.layers.0.self_attn.out_proj.bias
transformer_encoder.layers.0.linear1.weight
transformer_encoder.layers.0.linear1.bias
transformer_encoder.layers.0.linear2.weight
transformer_encoder.layers.0.linear2.bias
transformer_encoder.layers.0.norm1.weight
transformer_encoder.layers.0.norm1.bias
transformer_encoder.layers.0.norm2.weight
transformer_encoder.layers.0.norm2.bias
mlp_encoder.0.weight
mlp_encoder.0.bias
mlp_encoder.2.weight
mlp_encoder.2.bias
transformer_decoder.layers.0.self_attn.in_proj_weight
transformer_decoder.layers.0.self_attn.in_proj_bias
transformer_decoder.layers.0.self_attn.out_proj.weight
transformer_decoder.layers.0.self_attn.out_proj.bias
transformer_decoder.layers.0.multihead_attn.in_proj_weight
transformer_decoder.layers.0.multihead_attn.in_proj_bias
transformer_decoder.layers.0.multihead_a

In [24]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)